In [1]:
import json
import textdistance
import re
import joblib
from models.feature_extractor import FeatureExtractor
from models.win_rate_models import WinRateSciKitModel, WinRateSciKitModelConfidence
from models.mcts_draft import AllPickDraft, CaptainsModeDraft, getOrderedMoves
from mcts import mcts
import math

In [2]:
def simple_name(name):
    start = name.lower().split()[0]
    return re.sub(r'[^a-z]+', '', start)

In [3]:
def load_hero_names(file_path):
    with open(file_path, 'rt') as file:
        heroes = json.load(file)
        
    result = {}
    for hero in heroes.values():
        result[hero['id']] = (hero['localized_name'], simple_name(hero['localized_name']))
        
    return result
        
    
        

In [4]:
heroes = load_hero_names('input/heroes.json')

In [5]:
def get_most_confidence_hero(text, heroes):
    best_id = -1
    best_value = 0
    for hero_id,(localized_name, simple_name) in heroes.items():
        value = textdistance.levenshtein.normalized_similarity(text.lower(), simple_name)
        # print(value, localized_name)
        if value > best_value:
            best_value = value
            best_id = hero_id
    return (best_id, heroes[best_id][0])
        

In [6]:
get_most_confidence_hero('antymage',heroes)

(1, 'Anti-Mage')

In [7]:
def convert_id_to_str(values, heroes):
    return [heroes[val][0] for val in values] 

def print_state(bans, radiant_picks, dire_picks, heroes):
    p_val = {}
    p_val['bans'] = convert_id_to_str(bans, heroes)
    p_val['radiant_picks'] = convert_id_to_str(radiant_picks, heroes) 
    p_val['dire_picks'] = convert_id_to_str(dire_picks, heroes)
    print(p_val)

In [ ]:
radiant_picks = set()
dire_picks = set()
bans = set()

feature_extractor = joblib.load('input/feature_extractor.joblib')
linear_svc = joblib.load('input/linear_svc.joblib')
win_rate_model = WinRateSciKitModelConfidence(feature_extractor, linear_svc)

std_in = ''
while True:
    std_in = str(input(">"))
    part = std_in.split()
    if std_in == '':
        continue
    
    commad = part[0]
    values = part[1:]
    

    
    if commad == 'e':
        break
        
    if commad == 'mr':
        print('Start mcts radiant')
        state = AllPickDraft(win_rate_model, True, bans, radiant_picks, dire_picks)
        mcts_object = mcts(timeLimit=2_000, explorationConstant=1/math.sqrt(2))
        mcts_object.search(state)
        top_n = getOrderedMoves(mcts_object, 5)
        print(convert_id_to_str(top_n, heroes))
        
    if commad == 'md':
        print('Start mcts dire')
        state = AllPickDraft(win_rate_model, False, bans, radiant_picks, dire_picks)
        mcts_object = mcts(timeLimit=2_000, explorationConstant=1/math.sqrt(2))
        mcts_object.search(state)
        top_n = getOrderedMoves(mcts_object, 5)
        print(convert_id_to_str(top_n, heroes))
        
        
    edit_heroes = set()
    for value in values:
            hero_id = get_most_confidence_hero(value,heroes)[0]
            edit_heroes.add(hero_id)
    
    if commad == '+b':
        bans |= edit_heroes
    elif commad == '-b':
        bans -= edit_heroes
    elif commad == '+r':
        radiant_picks |= edit_heroes
    elif commad == '-r':
        radiant_picks -= edit_heroes
    elif commad == '+d':
        dire_picks |= edit_heroes
    elif commad == '-d':
        dire_picks -= edit_heroes
        
    print_state(bans, radiant_picks, dire_picks, heroes)


> +b morph


{'bans': ['Morphling'], 'radiant_picks': [], 'dire_picks': []}


> +b wrait


{'bans': ['Morphling', 'Wraith King'], 'radiant_picks': [], 'dire_picks': []}


> +r crys


...

Start mcts dire
['Axe', 'Bane', 'Bloodseeker', 'Drow Ranger', 'Earthshaker']
{'bans': ['Morphling', 'Wraith King'], 'radiant_picks': ['Crystal Maiden'], 'dire_picks': ['Ogre Magi']}


> +r dazz


{'bans': ['Morphling', 'Wraith King'], 'radiant_picks': ['Dazzle', 'Crystal Maiden'], 'dire_picks': ['Ogre Magi']}


> md


Start mcts dire
['Leshrac', 'Anti-Mage', 'Axe', 'Bane', 'Bloodseeker']
{'bans': ['Morphling', 'Wraith King'], 'radiant_picks': ['Dazzle', 'Crystal Maiden'], 'dire_picks': ['Ogre Magi']}
